In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import datetime as dt
from scipy.spatial.distance import cdist
from TBAG.TrajectoryClustering import TBAG
from sklearn.metrics.pairwise import haversine_distances
# from sklearn.cluster import DBSCAN, AgglomerativeClustering
# from sklearn.neighbors import KDTree
# from sklearn.preprocessing import RobustScaler

In [2]:
plots = pd.concat((pd.read_csv(r'blue_tracks_absolute.csv'), pd.read_csv(r'red_tracks_absolute.csv')), axis=0)

In [3]:
plots = plots.drop(['Unnamed: 0', 'BlueTrackId', 'RedTrackId'], axis=1)
plots['epoch'] = (pd.to_datetime(plots['TimeStamp']) - dt.datetime(1970, 1, 1)).dt.total_seconds()

In [4]:
plots[['Lat', 'Lon']] = np.deg2rad(plots[['Lat', 'Lon']])

In [5]:
plots = plots.sort_values('epoch').reset_index()

In [6]:
data = plots[['Lat', 'Lon', 'Alt', 'epoch']].to_numpy(dtype='float64')

In [7]:
# for i in plots[['Lat', 'Lon', 'Alt', 'epoch']].to_numpy():
#     print(i)

In [8]:
model = TBAG(eps=100, alpha=3000, max_speed=300, min_speed=0, window=4)
plots['clusterid'] = model.fit_predict(plots, lat_col='Lat', lon_col='Lon', alt_col='Alt', timestamp_col='epoch').astype('str')

KeyError: 'devsys'

In [ ]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='clusterid', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [ ]:
spatial_plots = RobustScaler().fit_transform(plots[['Lat', 'Lon', 'Alt', 'epoch']])

kdt = KDTree(spatial_plots, metric='euclidean')

k = 2 * spatial_plots.shape[-1] - 1

dist, ind = kdt.query(spatial_plots, k=k)

dist = np.sort(dist, axis=0)
dist = dist[:, k-1]
px.line(dist)

In [ ]:
model = DBSCAN(eps=0.1581433, min_samples=spatial_plots.shape[1] * 2)
plots['clusterid'] = model.fit_predict(spatial_plots).astype('str')

In [ ]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='clusterid', hover_name='TimeStamp', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [ ]:
def calc_angle(a, b):
    m, k = a.shape
    n, kk = b.shape

    result = np.zeros((m, n), dtype='float')

    if m < n:
        for i in range(m):
            delta = a[i] - b
            result[i, :] = np.arctan(delta[:, 1] / delta[:, 0])
    else:
        for j in range(n):
            delta = a - b[j]
            result[:, j] = np.arctan(delta[:, 1] / delta[:, 0])
    return result

In [ ]:
def distance_function(a):
    print(a.shape)
    euclidean_distance = cdist(a, a)
    angles = calc_angle(a, a)
    return euclidean_distance + angles

In [ ]:
model = AgglomerativeClustering(distance_threshold=0.0995433, linkage='single', n_clusters=None)
plots['clusterid'] = model.fit_predict(spatial_plots).astype('str')

In [ ]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='clusterid', hover_name='TimeStamp', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [93]:
plots['cluster_id'] = np.array([0, 1, 1, 0, 2, 1, 3, 1, 3, 1, 1, 2, 0, 1, 1, 2, 1, 0, 1, 3, 1, 2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 0, 2, 1, 0, 1, 1, 0, 0, 4, 2, 1, 2, 1, 1, 0, 1, 0, 1, 5, 0, 1, 1, 2, 0, 1, 0, 6, 1, 0, 2, 1, 1, 0, 7, 2, 0, 1, 1, 0, 2, 8, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 2, 1, 9, 1, 2, 1, 0, 0, 1, 1, 0, 1, 
2, 0, 1, 1, 0, 0, 1, 1, 2, 1, 2, 0, 10, 1, 0, 0, 1, 0, 1, 1, 2, 11, 0, 1, 2, 1, 0, 1, 0, 1, 12, 1, 0, 0, 1, 0, 1, 1, 0, 1, 2, 2, 1, 1, 0, 1, 0, 0, 1, 1, 2, 13, 2, 0, 1, 1, 2, 14, 1, 1, 0, 1, 2, 1, 0, 1, 0, 2, 1, 14, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 1, 15, 1, 0, 1, 2, 1, 1, 2, 
1, 0, 0, 1, 2, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 16, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 17, 2, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 0, 0, 18, 1, 0, 1, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 1, 19, 1, 2, 1, 0, 1, 0, 2, 1, 0, 1, 1, 2, 1, 0, 1, 20, 0, 1, 1, 2, 0, 1, 1, 20, 20, 1, 2, 1, 0, 1, 0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0]).astype('str')

In [ ]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='cluster_id', hover_name='TimeStamp', template='plotly_dark')
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()